In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re


#將被拆開的文字重新黏在一起，keyword is a list
def stick(k):
    blank = ""
    for i in range(len(k)):
        blank = blank + k[i]
    return blank

def writefile(s, pathway):
    with open(r'%s'%a,"pathway",encoding="utf-8") as f:
        s = str(s)
        f.write(s)
        
#清除<.*>，keyword is a string
def Remove(k2):
    fef = re.findall(r'<.*?>', k2)
    fef = set(fef)
    fef = list(fef)
    for i in range(len(fef)):
        k2 = k2.split(fef[i])
        k2 = stick(k2)

    return k2

def reptile1(keyword, number):
    a = keyword
    b = number
    #顯示不同搜尋結果個數的網址
    if 51 > int(b) > 25:
        url = 'https://www.sciencedirect.com/search/advanced?qs=%s&show=50'%a
    elif int(b) > 50:
        url = 'https://www.sciencedirect.com/search/advanced?qs=%s&show=100'%a 
    elif int(b) > 100 or int(b) <= 0:
        return 'wronginput'
    else:
        url = 'https://www.sciencedirect.com/search/advanced?qs=%s'%a
    
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
    resp = requests.get(url, headers=headers)
    resp.encoding = 'utf-8'
    raw_html = resp.text
    # 將 HTML 轉成 BeautifulSoup 物件，這裡使用 html.parser 內建解析器
    soup = BeautifulSoup(raw_html, 'html.parser')
    
    # soup.select 傳入使用 Chrome CSS Selector 選到對應的元素位置，取出裡面的值，組成字典，最後再放入list中
    s = []
    for i in range(1,int(b)+1,1):
        c = str(i)
        sdict = {'Paper title':"","Paper link":"","Paper download":""}
        paperTitle = soup.find_all(attrs={"data-aa-name": "srp-article-customer-was-entitled-with-entitled-rank-%s"%c})[0]
        paperDownload = soup.find_all(attrs={"data-aa-name": "srp-pdf-customer-was-entitled-with-entitled-rank-%s"%c})[0]
        paperDownload = str(paperDownload)
        paperTitle = str(paperTitle)
        
        #將title中的<em></em>去除
        k = re.findall(r'\">.*',paperTitle)[0][2:-4]
        ss = Remove(k)
        sdict['Paper title'] = ss
        sdict['Paper link'] = 'https://www.sciencedirect.com'+str(re.findall(r'href=\".*\"',paperTitle)[0].split('"')[1])
        sdict['Paper download'] = 'https://www.sciencedirect.com'+str(re.findall(r'href=\".*\"',paperDownload)[0].split('"')[1])
        
        s.append(sdict)
        time.sleep(0.5)
    return s

#抓摘要
def reptile2(s,paperNumber):
    #Find Abstract
    url = s[paperNumber]['Paper link']
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
    resp = requests.get(url, headers=headers)
    resp.encoding = 'utf-8'
    raw_html = resp.text
    # 將 HTML 轉成 BeautifulSoup 物件，這裡使用 html.parser 內建解析器
    soup = BeautifulSoup(raw_html, 'html.parser')
    # soup.select 傳入使用 Chrome CSS Selector 選到對應的元素位置，取出裡面的值，組成字典，最後再放入list中
    Abstract = soup.find_all(attrs={"class": "abstract author"})
    Abstract = str(Abstract)
    k = re.findall(r'Abstract.*', Abstract)[0][:-17]
    k2 = re.findall(r'>\w.*.*', k)[0][1:]
    
    abstract = Remove(k2)
    
    return abstract

keyword = input("please input one keyword:")
number = input("please input how many results you want to show (within 100):")

s = reptile1(keyword, number)
print('done')


please input one keyword:cordycep
please input how many results you want to show (within 100):12
done


In [ ]:
import tkinter as tk

# 使用模組
window = tk.Tk()
# 
window.title('Paper Searching')
# 設定畫面大小
window.geometry('800x600')
window.configure(background='white')

def breakABS(ABS):
    ABS = ABS.split(" ")
    xs = ""
    xss = []
    for j in range(len(ABS)//10):
        for i in range(10):
            xs =xs+" "+ABS[j*10+i]
        xss.append(xs)
        xs = ""
    xs =""
    for i in range(len(ABS)%10):
        xs = xs+" "+ABS[(len(ABS)//10)*10+i]
    xss.append(xs)
    
    return xss

def getEntry():
            
    def goback():
        titleListbox.delete(0, tk.END)
        for i in range(0,2*int(number),2):
            e = int(i/2)
            e = str(e)+ ".  " + NpT[int(i/2)]
            titleListbox.insert(i+1, e)
            titleListbox.insert(i+2, "")
            titleListbox.grid(row=10, column=0, sticky=tk.N+tk.W)

    def downloads():
        titleListbox.delete(0, tk.END)
        titleListbox.insert(1, "請透過下載網址下載(須連上校方網址)，如無法下載，請透過合法方式下載")
        titleListbox.insert(2, s[Entry]['Paper download'])
        titleListbox.grid(row=10, column=0, sticky=tk.N+tk.W)
        
    Entry = entry.get()
    Entry = int(Entry)
    titleListbox.delete(0, tk.END)
    entry.delete(0, tk.END)
    titleListbox.insert(1,s[Entry]['Paper title'])
    ABS = reptile2(s,Entry)
    ABSB = breakABS(ABS)
    entryBB = tk.Button(window, text='回上一頁', command = goback)
    entryBBB = tk.Button(window, text='下載文章', command = downloads)
    entryBB.grid(row=3, column=0, sticky='w') 
    entryBBB.grid(row=4, column=0, sticky='w')    
    for i in range(len(ABSB)):
        titleListbox.insert(i+3, ABSB[i])
        titleListbox.grid(row=7, column=0, sticky=tk.N+tk.W)

def title(i, d):
    title_label = tk.Label(title_frame, text=str(i)).pack(side='left')
    title_label = tk.Label(title_frame, text=d).pack(side='left')    

def plus1():
    i = 1
    return i

header_label = tk.Label(window, text='Input wanted paper number')
header_label.grid(row=0, column=0, sticky='w')
entry = tk.Entry()
entryB = tk.Button(window, text='顯示摘要', command = getEntry)
entry.grid(row=1, column=0, sticky=tk.N+tk.W)
entryB.grid(row=2, column=0, sticky='w')

header_label = tk.Label(window, text='Paper title')
header_label.grid(row=6, column=0, sticky='w')

title_frame = tk.Frame(window)
    # 向上對齊父元件    
title_frame.grid(row = number, column=0, sticky='w')

NpT = [] 
for i in range(len(s)):
    d = s[i]['Paper title']
    NpT.append(d)
    
titleListbox = tk.Listbox(width = 200)
titleListbox['height'] = 2*number

for i in range(0,2*int(number),2):
    e = int(i/2)
    e = str(e)+ ".  " + NpT[int(i/2)]
    titleListbox.insert(i+1, e)
    titleListbox.insert(i+2, "")
    titleListbox.grid(row=10, column=0, sticky=tk.N+tk.W)
    
window.mainloop()